In [1]:
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import import_ipynb
import sys
from torch.utils.data import DataLoader
import torch
from sklearn.metrics import accuracy_score


def train(Autoformer, Autoformer_optimizer,args,partition):
    train_loader = DataLoader(partition["train"],batch_size= args.batch_size,shuffle=False,drop_last=True)
    
    Autoformer.train()
    train_loss = 0.0
    for (x,y) in train_loader:
        Autoformer.zero_grad()
        Autoformer_optimizer.zero_grad()

        x = x.float().to(args.device) # 64,10 40
        y = y.float().squeeze().to(args.device)
        


        Autoformer_out = Autoformer(x)

        loss = args.loss_fn(Autoformer_out, y)

        loss.backward()

        Autoformer_optimizer.step() ## parameter 갱신
        train_loss += loss.item()

    train_loss = train_loss / len(train_loader)
    return Autoformer, train_loss

def validation(Autoformer, args, partition):
    val_loader = DataLoader(partition["val"],batch_size= args.batch_size,shuffle=False,drop_last=True)

    Autoformer.eval()
    val_loss = 0.0
    with torch.no_grad():
        for (x,y) in val_loader:

            x = x.float().to(args.device) # 64,10 40
            y = y.float().squeeze().to(args.device)

            Autoformer_out = Autoformer(x)
                        
            loss = args.loss_fn(Autoformer_out, y)      
            val_loss += loss.item()

    val_loss = val_loss / len(val_loader)
    return Autoformer, val_loss

def test(Autoformer, args, partition):
    test_loader = DataLoader(partition["test"],batch_size= args.batch_size,shuffle=False,drop_last=True)
    
    Autoformer.eval()
    ACC_metric = 0.0
    with torch.no_grad():
        for (x,y) in test_loader:

            x = x.float().to(args.device) # 64,10 40
            y = y.float().squeeze().to(args.device)

            Autoformer_out = Autoformer(x)

            output_ = torch.where(Autoformer_out >= 0.5, 1.0, 0.0)

            output_.requires_grad = True

            perc_y_pred = output_.cpu().detach().numpy()     
            perc_y_true =  y.cpu().detach().numpy()
            acc = accuracy_score(perc_y_true, perc_y_pred)

            ACC_metric += acc

    ACC_metric = ACC_metric / len(test_loader)
     
    return ACC_metric


In [2]:
#====== Argument initializtion ======#
import argparse
import torch.nn as nn
parser = argparse.ArgumentParser()
args = parser.parse_args("")

#=============== Device ===============#
args.device = 'cuda' if torch.cuda.is_available() else "cpu"

#================ Path ================#

args.save_file_path = "D:\\Autoformer_result"

#========= Base Hyperparameter =========#
args.batch_size = 32
args.lr = 0.00005
args.L2 = 0.00001
args.epoch = 100
args.dropout = 0.15
args.loss_fn = nn.BCELoss()

#===== Transformer Hyperparameter =====#
from models.Autoformer import Model as Autoformer

args.Autoformer = Autoformer
args.input_feature_size = 40

args.Autoformer_feature_size = 64

args.nhead = 4

args.nlayer = 1
args.ts_len = 10
args.target_len = 1
# trans_feature_size / trans_nhead => int 필수


In [3]:
## 실행파일
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import time
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import os
import csv
from Stock_dataloader_csv_ti import stock_csv_read
from Stock_Dataset import StockDataset


args.data_list = os.listdir(r"C:\Users\lab\Desktop\Autoformer\data\kdd17\price_long_50")

with open(args.save_file_path + '\\' + 'Autoformer_result_t.csv', 'w', encoding='utf-8', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(["model", "stock", "entire_exp_time",  "avg_test_ACC", "avg_test_ACC_std"])

    for data in args.data_list:
        est = time.time()

        stock = data.split('.')[0]

        setattr(args, 'symbol', stock)
        args.new_file_path = args.save_file_path + '\\' + "Autoformer_" + args.symbol
        os.makedirs(args.new_file_path)
        
        csv_read = stock_csv_read(data, args.ts_len,args.target_len)
        split_data_list = csv_read.cv_split()

        with open(args.new_file_path + '\\'+ str(args.symbol)+'test_acc_list' +'.csv', 'w',newline='') as alist:
            www = csv.writer(alist)
            www.writerow(["acc_list"])

            ACC_cv = []
            for i, data in enumerate(split_data_list):
                
                args.sp_ith = i

                args.split_file_path = args.new_file_path + "\\" + str(i) +"th_iter"
                os.makedirs(args.split_file_path)
                ## Model                 
                Autoformer = args.Autoformer()
                
                Autoformer.to(args.device)

                ##Optimizer
                Autoformer_optimizer = optim.Adam(Autoformer.parameters(), lr=args.lr, weight_decay=args.L2)

                ## training
                Train_losses = []
                Validation_losses = []
                for epoch in range(args.epoch):
                    ts = time.time()

                    trainset = StockDataset(data[0])
                    valset = StockDataset(data[1])
                    testset = StockDataset(data[2])


                    partition = {'train': trainset, 'val': valset, 'test': testset}     

                    Autoformer, train_loss = train(Autoformer,Autoformer_optimizer, args, partition)
                    Autoformer, validation_loss = validation(Autoformer, args, partition)


                    ## .state_dict() : model의 parameter(W)만을 저장하는것임 => 다시 불러올 때 모델의 파라미터를 알고있어야함
                    if len(Validation_losses) == 0:
                        torch.save(Autoformer.state_dict(), args.split_file_path + '\\' + str(epoch) +'_Autoformer' +'.pt')
                    elif min(Validation_losses) > validation_loss:
                        torch.save(Autoformer.state_dict(), args.split_file_path + '\\' + str(epoch) +'_Autoformer' +'.pt')
                    
                    Train_losses.append(train_loss)
                    Validation_losses.append(validation_loss)
                    
                    te = time.time()

                    print('Epoch {}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
                    .format(epoch, train_loss, validation_loss, te - ts))

                ## Test
                # state_dict로 저장했기 때문에 model의 hyperparameter를 불러와야함
                Autoformer = args.Autoformer()
                
                Autoformer.to(args.device)

                # Model_selection
                min_val_losses = Validation_losses.index(min(Validation_losses)) ## 10 epoch일 경우 0번째~9번째 까지로 나옴

                Autoformer.load_state_dict(torch.load(args.split_file_path + '\\' + str(min_val_losses) +'_Autoformer' + '.pt'))

                ACC = test(Autoformer, args, partition)
                www.writerow([ACC])
                print('ACC: {}'.format(ACC))

                with open(args.split_file_path + '\\'+ str(min_val_losses)+'Epoch_test_metric' +'.csv', 'w') as fd:
                    print('ACC: {}'.format(ACC), file=fd)

                result = {}

                result['train_losses'] = Train_losses
                result['val_losses'] = Validation_losses
                result['ACC'] = ACC

                eet = time.time()
                entire_exp_time = eet - est
                

                ## draw loss curve
                fig = plt.figure()
                plt.plot(result['train_losses'])
                plt.plot(result['val_losses'])
                plt.legend(['train_losses', 'val_losses'], fontsize=15)
                plt.xlabel('epoch', fontsize=15)
                plt.ylabel('loss', fontsize=15)
                plt.grid()
                plt.savefig(args.split_file_path + '\\' + 'fig' + '.png')
                plt.close(fig)
                ACC_cv.append(result['ACC'])

        ACC_cv_ar = np.array(ACC_cv)
        acc_avg = np.mean(ACC_cv_ar)
        acc_std = np.std(ACC_cv_ar)

        wr.writerow(["Autoformer", args.symbol, entire_exp_time, acc_avg, acc_std])


importing Jupyter notebook from Stock_Dataset.ipynb
Epoch 0, Loss(train/val) 0.72104/0.70229. Took 2.17 sec
Epoch 1, Loss(train/val) 0.70832/0.68070. Took 0.54 sec
Epoch 2, Loss(train/val) 0.69834/0.67846. Took 0.49 sec
Epoch 3, Loss(train/val) 0.68866/0.67931. Took 0.47 sec
Epoch 4, Loss(train/val) 0.68783/0.67646. Took 0.52 sec
Epoch 5, Loss(train/val) 0.68915/0.65943. Took 0.50 sec
Epoch 6, Loss(train/val) 0.69032/0.66870. Took 0.46 sec
Epoch 7, Loss(train/val) 0.67537/0.67149. Took 0.46 sec
Epoch 8, Loss(train/val) 0.67777/0.68436. Took 0.46 sec
Epoch 9, Loss(train/val) 0.67901/0.69217. Took 0.45 sec
Epoch 10, Loss(train/val) 0.67236/0.69436. Took 0.47 sec
Epoch 11, Loss(train/val) 0.66917/0.69533. Took 0.45 sec
Epoch 12, Loss(train/val) 0.67144/0.69117. Took 0.46 sec
Epoch 13, Loss(train/val) 0.66177/0.69645. Took 0.45 sec
Epoch 14, Loss(train/val) 0.67358/0.70038. Took 0.45 sec
Epoch 15, Loss(train/val) 0.66180/0.70256. Took 0.46 sec
Epoch 16, Loss(train/val) 0.65576/0.69831. Too